In [ ]:
import openai
import random
import json

# Configure OpenAI API key
openai.api_key = "your_openai_api_key"

# Define available actions
actions = [
    "move forward", "move backward", "turn left", "turn right",
    "pick up object", "place object", "navigate to", "stop",
    "inspect", "signal"
]

# Define visible objects in the environment
visible_objects = ["red ball", "blue cube", "green pyramid", "charging station", "door"]

# Random distance generator
def random_distance():
    return round(random.uniform(0.5, 10.0), 2)

# Prompt template for the OpenAI API
def generate_prompt(action, visible_object, distance):
    return f"""
The robot is capable of performing the following actions: {', '.join(actions)}.
The robot can see the following objects: {', '.join(visible_objects)}.
The robot is currently {distance} meters away from the {visible_object}.

You will create a training dataset entry to teach a small LLM to make decisions. The dataset will include:
1. Query: A question the user might ask the robot.
2. Answer: A text response with a verbal detailed plan for the robot to achieve the task.
3. Behavior Tree: A behavior tree representing the actions needed, or an empty string if none is required.

Generate:
- A plausible user query involving the action "{action}" and the object "{visible_object}".
- A detailed text response explaining the plan and identifying missing actions if applicable.
- A behavior tree that represents the solution.

Format the output as:
Query: ...
Answer: ...
Behavior Tree: ...
"""

# Function to generate a single dataset entry using the OpenAI API
def generate_entry():
    action = random.choice(actions)
    visible_object = random.choice(visible_objects)
    distance = random_distance()
    
    prompt = generate_prompt(action, visible_object, distance)
    
    # Call the OpenAI API
    response = openai.Completion.create(
        engine="text-davinci-004",  # Use the appropriate engine for your case
        prompt=prompt,
        max_tokens=500,
        temperature=0.7
    )
    
    # Parse the response into structured data
    try:
        output = response["choices"][0]["text"].strip()
        query, answer, behavior_tree = map(str.strip, output.split("\n", 2))
        return {
            "query": query.replace("Query: ", "").strip(),
            "answer": answer.replace("Answer: ", "").strip(),
            "behavior_tree": behavior_tree.replace("Behavior Tree: ", "").strip()
        }
    except Exception as e:
        print(f"Error parsing response: {e}")
        return None

# Generate the full dataset
def generate_dataset(num_samples=100):
    dataset = []
    for _ in range(num_samples):
        entry = generate_entry()
        if entry:
            dataset.append(entry)
    return dataset

# Save the dataset to a JSON file
def save_dataset(dataset, filename="robot_training_dataset.json"):
    with open(filename, "w") as f:
        json.dump(dataset, f, indent=4)

# Generate and save the dataset
if __name__ == "__main__":
    num_samples = 100  # Adjust the number of samples as needed
    dataset = generate_dataset(num_samples)
    save_dataset(dataset)
    print(f"Dataset saved to robot_training_dataset.json")
